In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import ast

In [236]:
with open('COCO/keypoints.json', 'r') as f:
    keypoints_list = json.load(f)

In [237]:
hpe_attrs_list = ['nose', 'chest', 'right_shoulder', 'right_elbow', 'right_wrist', 'left_shoulder', 'left_elbow', 
              'left_wrist', 'right_hip', 'right_knee', 'right_ankle', 'left_hip', 'left_knee', 'left_ankle',
              'right_eye', 'left_eye', 'right_ear', 'left_ear']

In [238]:
coco_attrs_list = ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 
                   'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle']

In [239]:
main_list = []
for img in keypoints_list:
    img_id = img['Img_ID']
    for index, person in enumerate(img['people']):
        person_obj = {}
        person_obj['Img_ID'] = img_id
        person_obj['ID'] = f'{img_id}{index+1}'
        for i, attr in enumerate(hpe_attrs_list):
            person_obj[f'{attr}_x'] = person[i]['x']
            person_obj[f'{attr}_y'] = person[i]['y']
            person_obj[f'{attr}_score'] = person[i]['score']
        main_list.append(person_obj)

In [240]:
main_df = pd.DataFrame(main_list)

In [241]:
anns_df = pd.read_csv('../data/coco_diff/single_anns/person_anns.csv')
img_list = set(main_df['Img_ID'])

In [242]:
mini_anns_df = anns_df[anns_df['Img'].astype(str).isin(img_list)]
mini_anns_df = mini_anns_df[['Img', 'person_num', 'num_people', 'keypoints', 'img_size', 'area_norm', 'offset', 'vis_face_ratio', 'vis_body_ratio', 'vis_extremities_ratio']]

In [243]:
def keypoints_handling(row):
    keypoints_list = ast.literal_eval(row['keypoints'])
    for index, attr in enumerate(coco_attrs_list):
        row[f'{attr}_x'] = keypoints_list[index*3]
        row[f'{attr}_y'] = keypoints_list[index*3+1]
        row[f'{attr}_v'] = keypoints_list[index*3+2]        
    return row

mini_anns_df = mini_anns_df.apply(keypoints_handling, axis=1)